In [ ]:
%matplotlib notebook 
%reload_ext autoreload
%autoreload 2

In [ ]:
# general imports
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# MNE, spetral power functions, & FOOOF
import mne
from fooof import FOOOF
from neurodsp.spectral import compute_spectrum

# import statsmodels
import statsmodels.api as sm
from scipy.stats import wilcoxon, fisher_exact, chisquare

# import local functions from iEEG_LKM library
sys.path.append('/Users/christinamaher/Documents/Github/iEEG_LKM/preprocessing_utils.py')
sys.path.append('/Users/christinamaher/Documents/Github/iEEG_LKM/stats_utils.py')
from preprocessing_utils import join_clean_segments, load_label_file
from stats_utils import assign_band, BOSC_tf, BOSC_detect, average_power, peak_count, average_duration

# Initialize the directories
data_dir = '/Users/christinamaher/Documents/projects/ieeg_lkm/data'
results_dir = '/Users/christinamaher/Documents/projects/ieeg_lkm/results'

subject_list = ['sub-01','sub-02','sub-03','sub-04','sub-05','sub-06','sub-07','sub-08'] # Define the subject list 
conditions = ["baseline","meditation"] # Define the conditions
regions = ["amygdala","hippocampus"] # Regions 
sr = 250 # Define the sampling rate (250 Hz) for all recordings

# Preprocess iEEG data

In the next cell, we will loop through each subject's data by condition. We will visually inspect the data and annotate noise/IED based on clinical criterion: [Mercier et al. (2022)](https://www.sciencedirect.com/science/article/pii/S1053811922005559) ; [Marcuse et al. (2015)](https://www.sciencedirect.com/book/9780323353878/rowans-primer-of-eeg). Clean data will be saved in .fif format for further analyses.

In [ ]:
# Manually inspect data for each recording (subject x condition) and annotate noisy timpoints
for subject in subject_list:
    for condition in conditions:
        # Load the data
        raw_data = mne.io.read_raw_fif(data_dir + '/' + subject + '_' + condition + '_raw.fif', preload=True, verbose=False) # Load the data
        num_channels = raw_data.info['nchan'] # Get the number of channels
        fig = raw_data.plot(start=2, duration=50, n_channels=num_channels,scalings=raw_data._data.max()* 10) # Plot the data (all channels, 50 seconds per window)
        fig.fake_keypress("a")

In [ ]:
# Save annotated data to results directory 
for subject in subject_list:
    for condition in conditions:
        # save annotated data 
        raw_data.save(f'{results_dir}/{subject}_{condition}_annotated.fif', overwrite=True) # Save the annotated data for FOOOF and eBOSC

In [ ]:
# Drop noisy timepoints from data and save clean data to results directory 
for subject in subject_list:
    for condition in conditions:
        # load the annotated data
        annotated_data = mne.io.read_raw_fif(results_dir + '/' + subject + '_' + condition + '_annotated.fif', preload=True, verbose=False) 
        # concatenate good segments only 
        clean_data = join_clean_segments(annotated_data)
        # save clean data
        clean_data.save(f'{results_dir}/{subject}_{condition}_clean.fif', overwrite=True) # Save the cleaned data for FOOOF and eBOSC

In [ ]:
# Use chi-squared test to test whether the proportion of data retained following manual artifact rejection is significantly different than expected between conditions. 
baseline = []
meditation = []

for subject in subject_list:
    baseline_data = mne.io.read_raw_fif(results_dir + '/' + subject + f'_{conditions[0]}_clean.fif', preload=True, verbose=False) # Load the data
    baseline.append(baseline_data.n_times / (baseline_data.info['sfreq'] * 60))  # Get the proportion of clean data for the baseline condition (convert to minutes)
    meditation_data = mne.io.read_raw_fif(results_dir + '/' + subject + f'_{conditions[1]}_clean.fif', preload=True, verbose=False) # Load the data
    meditation.append(meditation_data.n_times / (meditation_data.info['sfreq'] * 60)) # Get the proportion of clean data for the meditation condition (convert to minutes)


observed_counts = np.array([np.sum(baseline), np.sum(meditation)]) # Calculate the mean proportion of clean data for each condition
expected_counts = np.array([np.sum(observed_counts)/2, np.sum(observed_counts)/2]) # Calculate the expected counts (assuming 50-50 distribution of clean data between conditions)

# Perform the chi-squared test
print(observed_counts, expected_counts)
chi2_stat, p = chisquare(f_obs=observed_counts, f_exp=expected_counts)

# Print results
print(f"Observed counts: {observed_counts}")
print(f"Expected counts: {expected_counts}")
print(f"Chi-squared test: stat = {chi2_stat:.2f}, p = {p:.4f}")

# FOOOF

In the following cell, we will loop through all patients by condition and perform [FOOOF model fit](https://github.com/fooof-tools/fooof). Results (aperiodic and periodic features of the power spectra for each channel x condition) will be saved to a csv file to perform statistics and visualization. 

In [ ]:
# Initialize lists to store the aperiodic and periodic parameters for each channel 
aperiodic_df = []
periodic_df = []

# Intialize FOOOF object
freq_range = [2, 55]
fm = FOOOF(aperiodic_mode='knee',peak_width_limits=[1, 8]) # hyperparameter setting

# Define bands dictionary to assign canonical frequency bands (str) to each frequency (Hz). 
bands = {
    'delta': (2, 4),
    'theta': (4, 8),
    'alpha': (8, 13),
    'beta': (13, 30),
    'gamma': (30, 55)
}

for subject in subject_list: # Loop through each subject
    baseline_data = mne.io.read_raw_fif(f'{data_dir}/{subject}_baseline_clean.fif') # Load the baseline data
    meditation_data = mne.io.read_raw_fif(f'{data_dir}/{subject}_meditation_clean.fif') # Load the meditation data
    num_channels = len(baseline_data.ch_names) # Get the number of channels in the data (this will be used to loop through each channel)
    sr = baseline_data.info['sfreq'] # sampling rate (250 Hz)

    for channel in range(num_channels): # Loop through each channel
        for condition in conditions: # Loop through each condition (baseline and meditation)
            data = globals()[f"{condition}_data"].get_data()[channel, :] # Get the data for the channel and condition
            freq_mean, psd_mean = compute_spectrum(data, sr, method='welch',nperseg=sr*2) # Compute the power spectrum for the data using Welch's method
            fm.report(freq_mean, psd_mean, freq_range) # Fit the power spectrum with the FOOOF model
            ap_params, peak_params, _, _, _ = fm.get_results() # Get the aperiodic and periodic parameters from the FOOOF model

            # Store the aperiodic/periodic parameters in a dataframe
            ap_params = ap_params.tolist() + [channel] + [condition] + [subject] # Add the channel number, condition, and subject info to the aperiodic parameters
            ap_params = np.reshape(ap_params, (1,6))
            columns = ['offset','knee','exponent', 'electrode','condition', 'subject'] # Define the columns for the aperiodic dataframe
            ap_df = pd.DataFrame(ap_params, columns=columns) # Create a dataframe from the aperiodic parameters
            aperiodic_df.append(ap_df) # Append the aperiodic dataframe to the list of aperiodic dataframes

            pp_df = pd.DataFrame(peak_params) # Create a dataframe from the periodic parameters
            pp_df['electrode'] = channel # Add the channel number to the peak_params dataframe
            pp_df = pp_df.rename(columns={0: 'center_frequency', 1: 'power', 2: 'bandwidth', 'electrode': 'elec'}) # Rename the columns of the peak_params dataframe
            pp_df['condition'] = condition # Add the condition to the peak_params dataframe
            pp_df['subject'] = subject # Add the subject to the peak_params dataframe
            periodic_df.append(pp_df) # Append the periodic dataframe to the list of periodic dataframes

# Concatenate the aperiodic and periodic dataframes
aperiodic_df = pd.concat(aperiodic_df) # Concatenate the list of aperiodic dataframes
periodic_df = pd.concat(periodic_df) # Concatenate the list of periodic dataframes

# Assign frequency bands to the center frequency of each peak
periodic_df['Freq_Band'] = periodic_df['center_frequency'].apply(assign_band,bands=bands)

# Save the aperiodic and periodic dataframes to a csv file
aperiodic_df.to_csv(f'{results_dir}/aperiodic_results.csv', index=False) 
periodic_df.to_csv(f'{results_dir}/periodic_results.csv', index=False)

In [ ]:
# Compute average power and number of peaks in each frequency band for each channel and condition using average_power() and peak_count() from stats_utils

# Group by unique combinations of electrode, subject, and condition
grouped = periodic_df.groupby(['elec', 'subject','condition'])

# Initialize an empty dictionary to store the results
avg_power_by_band = {}
num_peaks_by_band = {}

# Iterate over each group
for (elec, subject, condition), group_df in grouped:
    # Apply the average_power function to the group
    avg_power_by_band[(elec, subject, condition)] = average_power(group_df)
    num_peaks_by_band[(elec, subject, condition)] = peak_count(group_df,bands)
    
# Convert dictionary to DF
peaks_df = pd.DataFrame.from_dict(num_peaks_by_band, orient='index')
power_df = pd.DataFrame.from_dict(avg_power_by_band, orient='index')

# Reset index and rename columns
peaks_df.reset_index(inplace=True)
peaks_df.rename(columns={'level_0': 'electrode', 'level_1': 'subject', 'level_2': 'condition'}, inplace=True)
peaks_df = pd.melt(peaks_df, id_vars=['electrode', 'subject', 'condition'], var_name='freq_band', value_name='peak')

power_df.reset_index(inplace=True)
power_df.rename(columns={'level_0': 'electrode', 'level_1': 'subject', 'level_2': 'condition'}, inplace=True)
power_df = pd.melt(power_df, id_vars=['electrode', 'subject', 'condition'], var_name='freq_band', value_name='power')


# Add anatomical localization information to results dataframes 

# Loop through each subject
for subject in subject_list:
    # Load label file for the current subject
    label_df = load_label_file(subject)
    
    # If label file exists
    if label_df is not None:
        # Merge anatomical localization information with power and peak dataframes
        power_df_subject = power_df[power_df['subject'] == subject]
        peaks_df_subject = peaks_df[peaks_df['subject'] == subject]
        
        power_df_subject = pd.merge(power_df_subject, label_df, how='left', on='electrode')
        peaks_df_subject = pd.merge(peaks_df_subject, label_df, how='left', on='electrode')
        
        # Append merged dataframes to the list
        all_power_dfs.append(power_df_subject)
        all_peaks_dfs.append(peaks_df_subject)

# Concatenate dataframes for all subjects
all_power_df = pd.concat(all_power_dfs, ignore_index=True)
all_peaks_df = pd.concat(all_peaks_dfs, ignore_index=True)

# Save results to a csv file 
power_df.to_csv(f'{results_dir}/power_df.csv', index=False)
peaks_df.to_csv(f'{results_dir}/peaks_df.csv', index=False)


In [ ]:
# Use wilcoxon signed-rank test to compare the average power in each frequency band between conditions. Based on literature and hypotheses, tests for gamma band are one-tailed, while tests for other bands are two-tailed.

for region in regions:
    for band in bands.keys(): # Loop through each frequency band
        baseline_power = power_df.query(f"condition == 'baseline' & freq_band == '{band}' & region == '{region}'")['power'] # Get power in the baseline condition for the band
        meditation_power = power_df.query(f"condition == 'meditation' & freq_band == '{band}' & region == '{region}'")['power'] # Get power in the meditation condition for the band

        # Filter out rows with null values in either baseline or meditation power
        valid_indices = ~(baseline_power.isnull() | meditation_power.isnull())
        baseline_power = baseline_power[valid_indices]
        meditation_power = meditation_power[valid_indices]

        if band == 'gamma': # Check if the band is gamma
            stat, p = wilcoxon(baseline_power, meditation_power, alternative='greater') # Perform the Wilcoxon signed-rank test with a one-tailed test
        else: # If the band is not gamma
            stat, p = wilcoxon(baseline_power, meditation_power) # Perform the Wilcoxon signed-rank test with a two-tailed test
        print(f"{region} {band}: stat = {stat}, p = {p}") # Print the p-value

In [ ]:
# calculate proportion of peaks by band, region, and condition
prop_peaks_by_band = peaks_df.groupby(['condition', 'freq_band','region'])['peak'].mean().reset_index()

# Use Fisher's exact test to compare the proportion of peaks detected in each frequency band between conditions

# Group the data by frequency band and region
grouped_data = prop_peaks_by_band.groupby(['freq_band','region'])

# Perform Fisher's exact test for each frequency band and region combination
for (freq_band,region), freq_band_region_data in grouped_data:
    contingency_table = [[freq_band_region_data[(freq_band_region_data['condition'] == 'baseline')]['peak'].sum(), freq_band_region_data[(freq_band_region_data['condition'] == 'meditation')]['peak'].sum()],
                         [len(freq_band_region_data[(freq_band_region_data['condition'] == 'baseline')]) - freq_band_region_data[(freq_band_region_data['condition'] == 'baseline')]['peak'].sum(),
                          len(freq_band_region_data[(freq_band_region_data['condition'] == 'meditation')]) - freq_band_region_data[(freq_band_region_data['condition'] == 'meditation')]['peak'].sum()]]

    oddsratio, p = fisher_exact(contingency_table)
    print(f"Fisher's exact test for {freq_band} in {region}: p-value = {p}")

# eBOSC

In the following cell, we will loop through all patients by condition and perform [eBOSC model fit](https://github.com/jkosciessa/eBOSC_py). Results (proportion of time spent oscillating at each frequency for each channel x condition) will be saved to a csv file to perform statistics and visualization. 

In [ ]:
results = [] # Initialize a list to store the results (one dataframe per channel: columns are frequency (int), frequency band (str), proportion of time spent oscillating (int), channel (str), participant (str), and condition (str))

for subject in subject_list: # Loop through each subject

    baseline_data = mne.io.read_raw_fif(f'{data_dir}/{subject}_baseline_clean.fif') # Load the baseline data
    meditation_data = mne.io.read_raw_fif(f'{data_dir}/{subject}_meditation_clean.fif') # Load the meditation data
    num_channels = len(baseline_data.ch_names) # Get the number of channels in the data (this will be used to loop through each channel)
    sr = baseline_data.info['sfreq'] # Sampling rate - this is the same for both baseline and meditation data (250 Hz for these NeuroPace data).

    for channel in range(num_channels): # Loop through each channel
        for condition in conditions: # Loop through each condition (baseline and meditation)
            data = globals()[f"{condition}_data"].get_data()[channel, :] # Get the data for the channel and condition
            f = np.linspace(1, 55, 44) # Frequency range - this is the same for both baseline and meditation data (1-55 Hz). 
            wavelet = 4 # the wavenumber to use for the Morlet wavelet
            B, T, F = BOSC_tf(data, f, sr, wavelet) # Compute the BOSC time-frequency matrix for a given LFP signal.
            exog = np.log10(F) # Log transform the frequency values
            exog = sm.add_constant(exog) # Add a constant to the exogenous variable
            endog = np.log10(B).mean(axis=-1) # Log transform the BOSC values and take the mean across time
            rlm_model = sm.RLM(endog, exog, M=sm.robust.norms.TukeyBiweight()) # Create a robust linear model which is used to fit the data because it is less sensitive to outliers than OLS regression. 
            rlm_results = rlm_model.fit() # Fit the model
            pv = np.zeros(2) # Initialize the parameter vector
            pv[0] = rlm_results.params[1] # Assign the slope to the first element of the parameter vector
            pv[1] = rlm_results.params[0] # Assign the intercept to the second element of the parameter vector
            mp = 10**(np.polyval(pv,np.log10(F))) # Compute the model fit
            pt=chi2.ppf(0.95,2)*mp/2 # Compute the power threshold (95% confidence interval for the model fit)
            dt=(3*sr/F) # Compute the duration threshold (3 cycles of the given frequency)
            detected = np.zeros_like(B) # Initialize the detected array
            for freq_ix in range(len(F)): # Loop through each frequency
                detected[freq_ix, :] = BOSC_detect(B[freq_ix,:], # Detect oscillations based on the power timecourse as a given frequency using pre-defined power and duration thresholds
                                                powthresh=pt[freq_ix], # Power threshold
                                                durthresh=dt[freq_ix], # Duration threshold
                                                Fsample=sr) # Sampling rate
            time_spent_oscillating = detected.sum(axis=-1) # Compute the time spent oscillating
            total_time = len(T) # Compute the total time
            proportion_time_oscillating = time_spent_oscillating/total_time # Compute the proportion of time spent oscillating (time spent oscillating/total time)
            temp_df = pd.DataFrame({'Freq': F, 'Prop_Time': proportion_time_oscillating, 'Channel':channel, 'Participant':subject,'Condition':condition}) # Create a temporary dataframe to store the results from the current channel and condition
            results.append(temp_df) # Append the temporary dataframe to the results list
        
results = pd.concat(results,ignore_index=True) # Concatenate the results list into a single dataframe


# Assign frequency bands to the frequencies
results['Freq_Band'] = results['Freq'].apply(assign_band, bands=bands) # Apply the function to the 'Freq' column to create a new column 'FrequencyBand' that assigns the canonical frequency band based on the frequency. 

# Save the dataframe to a csv file
results.to_csv(f'{results_dir}/ebosc_results.csv', index=False) 

In [ ]:
# Let's compute the average proportion of time spent oscillating in each frequency band for each channel and condition using average_duration() from stats_utils
avg_duration_by_band = average_duration(results)

# Save results to a csv file 
avg_duration_by_band.to_csv(f'{results_dir}/avg_duration_by_band.csv', index=False)

In [ ]:
# Use wilcoxon signed-rank test to compare the average proportion of time spent oscillating in each frequency band between conditions. Based on literature and hypotheses, tests for beta band are one-tailed, while tests for other bands are two-tailed.
for region in regions:
    for band in bands.keys(): # Loop through each frequency band
        baseline_duration = avg_duration_by_band.query(f"condition == 'baseline' & band == '{band}' & region == '{region}'")['duration'] # Get the average duration in the baseline condition for the band
        meditation_duration = avg_duration_by_band.query(f"condition == 'meditation' & band == '{band}' & region == '{region}'")['duration'] # Get the average duration in the meditation condition for the band
        if band == 'beta':
            stat, p = wilcoxon(baseline_duration, meditation_duration, alternative='greater') # Perform the Wilcoxon signed-rank test with a one-tailed test
        else:
            stat, p = wilcoxon(baseline_duration, meditation_duration) # Perform the Wilcoxon signed-rank test with a two-tailed test

        print(f"{region} {band}: stat = {stat}, p = {p}") # Print the p-value

# Supplemental analyses 

In [ ]:
# Let's compare aperiodic parameters between regions (amygdala and hippocampus) at baseline

# Load the aperiodic results
aperiodic_df = pd.read_csv(f'results_dir/aperiodic_results.csv')

# Filter the aperiodic results to only include the amygdala and hippocampus
amygdala = aperiodic_df.query("region == 'amygdala'") # Filter the aperiodic results to only include the amygdala
hippocampus = aperiodic_df.query("region == 'hippocampus'") # Filter the aperiodic results to only include the hippocampus

# Perform a wilcoxon signed-rank test to compare the aperiodic parameters between the amygdala and hippocampus at baseline

for param in ['offset','knee','exponent']: # Loop through each aperiodic parameter
    amygdala_param = amygdala.query("condition == 'baseline'")[param] # Get the aperiodic parameter for the amygdala at baseline
    hippocampus_param = hippocampus.query("condition == 'baseline'")[param] # Get the aperiodic parameter for the hippocampus at baseline
    stat, p = wilcoxon(amygdala_param, hippocampus_param) # Perform the Wilcoxon signed-rank test
    print(f"{param}: stat = {stat}, p = {p}") # Print the p-value

In [ ]:
# Let's compare periodic parameters (power, duration) between regions (amygdala and hippocampus) at baseline

# Load the average power by band results
power_df = pd.read_csv(f'results_dir/avg_power_by_band.csv')
power_df['periodic_feature'] = 'power' 
duration_df = pd.read_csv(f'results_dir/avg_duration_by_band.csv')
duration_df['periodic_feature'] = 'duration'

# Combine the power and duration dataframes
periodic_df = pd.merge(power_df, duration_df, on=['electrode','condition','subject','band']) # Merge the power and duration dataframes

# Filter the periodic results to only include the amygdala and hippocampus
amygdala = periodic_df.query("region == 'amygdala'") # Filter the periodic results to only include the amygdala
hippocampus = periodic_df.query("region == 'hippocampus'") # Filter the periodic results to only include the hippocampus

# Perform a wilcoxon signed-rank test to compare the periodic parameters between the amygdala and hippocampus at baseline

for param in ['power','duration']: # Loop through each periodic parameter
    amygdala_param = amygdala.query("condition == 'baseline'")[param] # Get the periodic parameter for the amygdala at baseline
    hippocampus_param = hippocampus.query("condition == 'baseline'")[param] # Get the periodic parameter for the hippocampus at baseline
    stat, p = wilcoxon(amygdala_param, hippocampus_param) # Perform the Wilcoxon signed-rank test
    print(f"{param}: stat = {stat}, p = {p}") # Print the p-value

In [ ]:
# Let's compare the magnitude of condition-related changes (meditation - baseline) in periodic parameter (power, duration) between regions (amygdala and hippocampus)

# Compute the change in periodic parameters between conditions
periodic_df['change'] = periodic_df['meditation'] - periodic_df['baseline']

# Filter the periodic results to only include the amygdala and hippocampus
amygdala = periodic_df.query("region == 'amygdala'") # Filter the periodic results to only include the amygdala
hippocampus = periodic_df.query("region == 'hippocampus'") # Filter the periodic results to only include the hippocampus

# Perform a wilcoxon signed-rank test to compare the change in periodic parameters between the amygdala and hippocampus

for param in ['power','duration']: # Loop through each periodic parameter
    amygdala_param = amygdala.query("condition == 'baseline'")[param] # Get the change in the periodic parameter for the amygdala
    hippocampus_param = hippocampus.query("condition == 'baseline'")[param] # Get the change in the periodic parameter for the hippocampus
    stat, p = wilcoxon(amygdala_param, hippocampus_param) # Perform the Wilcoxon signed-rank test
    print(f"{param}: stat = {stat}, p = {p}") # Print the p-value